<a href="https://colab.research.google.com/github/Abeeba540/Amazon-Sales-Analysis-Dashboard/blob/main/Solve_Business_Problems_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prerequisites**

In [1]:
INPUT_XLSX_PATH = "/mnt/data/Solving Business Problems with AI.xlsx"


## **Configure OpenAI API Key.**

In [2]:
# Install the OpenAI Python package.
%pip install openai httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.52.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [4]:
from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='a0Bfv000005pwFBEAY' # Replace with your actual API key
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [5]:
import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [7]:
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# Authenticate with Google and create the gspread client object 'gc'
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

output_document = gc.create('Solving Business Problems with AI - Output')

# Create 'email-classification' sheet
email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')

# Example of writing the data into the sheet
# Assuming you have your classification in the email_classification_df DataFrame
# set_with_dataframe(email_classification_sheet, email_classification_df)
# Or directly update cells: https://docs.google.com/spreadsheets/d/en/latest/user-guide.html#updating-cells

# Create 'order-status' sheet
order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

# Create 'order-response' sheet
order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Create 'inquiry-response' sheet
inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')

# This is the solution output link, paste it into the submission form
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")

Shareable link: https://docs.google.com/spreadsheets/d/1jlEYV-PyjxJ_gjRBPVSYuoxTAM5SO9BtE-1Wq5ExiSM


# **Task 1. Classify emails**

In [15]:
# Inspect
print("Products columns:", products_df.columns.tolist())
print("Emails columns:", emails_df.columns.tolist())
display(products_df.head(2))
display(emails_df.head(2))

# Normalize and create lookups using actual column names seen above
products_df.columns = [c.strip() for c in products_df.columns]
emails_df.columns = [c.strip() for c in emails_df.columns]

# choose the correct column names from the printout, example below uses product_id & stock
if 'product_id' in products_df.columns:
    products_df['product_id'] = products_df['product_id'].astype(str)
else:
    # fallback: if it's product_id with different name, adapt here
    raise KeyError("Expected 'product_id' in products_df columns. Check the printed column names above.")

product_lookup = products_df.set_index('product_id').to_dict(orient='index')

# Build stock dict (adjust 'stock' to the exact column name if different)
stock = {str(row['product_id']): int(row.get('stock', 0)) for _, row in products_df.iterrows()}

print("Sample product lookup entry:", list(product_lookup.items())[:1])
print("Sample stock dict entry:", list(stock.items())[:3])


Products columns: ['product_id', 'name', 'category', 'description', 'stock', 'seasons', 'price']
Emails columns: ['email_id', 'subject', 'message']


,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."


Sample product lookup entry: [('RSG8901', {'name': 'Retro Sunglasses', 'category': 'Accessories', 'description': "Transport yourself back in time with our retro sunglasses. These vintage-inspired shades offer a cool, nostalgic vibe while protecting your eyes from the sun's rays. Perfect for beach days or city strolls.", 'stock': 1, 'seasons': 'Spring, Summer', 'price': 26.99})]
Sample stock dict entry: [('RSG8901', 1), ('SWL2345', 5), ('VSC6789', 6)]


In [17]:
# Task 1: classify emails — corrected to handle ChatCompletionMessage objects
import json, time

def _get_message_text(resp):
    """
    Robust accessor for the response text:
    - Works if the SDK returns a ChatCompletionMessage object (has .content)
    - Works if it returns a dict-like structure (['message']['content'])
    """
    try:
        # preferred: message is an object with .content
        return resp.choices[0].message.content
    except Exception:
        try:
            # fallback: older dict-like shape
            return resp['choices'][0]['message']['content']
        except Exception:
            # fallback: other maybe text field
            return str(resp)

def classify_email_with_llm(subject: str, body: str) -> str:
    prompt = (
        "Classify the following customer email into one of two categories: "
        "'product inquiry' or 'order request'. Respond with only the category string.\n\n"
        f"Subject: {subject}\n\nBody: {body}\n\nCategory:"
    )
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role":"user","content":prompt}],
        max_tokens=8,
        temperature=0.0,
    )
    out = _get_message_text(resp).strip().lower()
    if "order" in out:
        return "order request"
    return "product inquiry"

# Quick test on the first email to avoid spending tokens
test_row = emails_df.iloc[0]
print("Testing classification on first email:", test_row['email_id'])
print("Subject:", test_row['subject'])
print("Message:", test_row['message'][:200], "...")
print("LLM classification:", classify_email_with_llm(test_row['subject'], test_row['message']))


Testing classification on first email: E001
Subject: Leather Wallets
Message: Hi there, I want to order all the remaining LTH0976 Leather Bifold Wallets you have in stock. I'm opening up a small boutique shop and these would be perfect for my inventory. Thank you! ...
LLM classification: order request


In [18]:
classification_rows = []
for _, r in emails_df.iterrows():
    eid = r['email_id']
    subj = r.get('subject','')
    msg = r.get('message','')
    try:
        cat = classify_email_with_llm(subj, msg)
    except Exception as e:
        cat = "product inquiry"   # safe default
        print("LLM classify error for", eid, "->", e)
    classification_rows.append({'email ID': eid, 'category': cat})
    time.sleep(0.2)

email_classification_df = pd.DataFrame(classification_rows)
display(email_classification_df)


,email ID,category
0,E001,order request
1,E002,order request
2,E003,product inquiry
3,E004,order request
4,E005,product inquiry
5,E006,product inquiry
6,E007,order request
7,E008,order request
8,E009,product inquiry
9,E010,order request


In [21]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=email_classification_df)

https://docs.google.com/spreadsheets/d/1eXBENs-e37Q4ARKhXfgmlqULYRdWOCbPBeGNKNgA6VI/edit#gid=0


In [22]:
def _get_message_text(resp):
    """
    Robust accessor for the response text:
    - Works if SDK returns a ChatCompletionMessage object (has .content)
    - Works if it returns a dict-like structure (['choices'][0]['message']['content'])
    """
    try:
        return resp.choices[0].message.content
    except Exception:
        try:
            return resp['choices'][0]['message']['content']
        except Exception:
            # last-resort stringification
            return str(resp)


# **Task 2. Process order requests**

In [24]:
# Task 2: extract ordered items and process stock (uses _get_message_text)
import json, time
from typing import List, Dict

def extract_order_items_with_llm(body: str) -> List[Dict]:
    prompt = (
        "Extract ordered items from this email. Return ONLY a valid JSON array of objects "
        "with keys 'product_id' or 'product_name' and 'quantity'. Example:\n"
        '[{"product_id":"RSG8901","quantity":2},{"product_name":"Sleek Wallet","quantity":1}]\n\n'
        f"Email:\n{body}\n\nOutput JSON:"
    )
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role":"user","content":prompt}],
        max_tokens=300,
        temperature=0.0,
    )
    text = _get_message_text(resp).strip()
    try:
        parsed = json.loads(text)
        if isinstance(parsed, dict):
            parsed = [parsed]
    except Exception:
        # fallback: try to extract JSON substring
        try:
            import re
            m = re.search(r'(\[.*\])', text, flags=re.S)
            parsed = json.loads(m.group(1)) if m else []
        except Exception:
            parsed = []
    return parsed

# Prepare mutable stock map using your products_df columns
stock = {str(r['product_id']): int(r.get('stock', 0)) for _, r in products_df.iterrows()}

order_status_rows = []
order_response_rows = []

# Process only emails classified as order requests
for _, r in emails_df.iterrows():
    eid = r['email_id']
    subj = r.get('subject','')
    msg = r.get('message','')
    # safe lookup in classification DF (it uses 'email ID' column)
    cat_row = email_classification_df.loc[email_classification_df['email ID']==eid, 'category'].values
    cat = cat_row[0] if len(cat_row) else classify_email_with_llm(subj, msg)

    if cat != "order request":
        continue

    items = extract_order_items_with_llm(msg)
    if not items:
        order_response_rows.append({
            'email ID': eid,
            'response': (
                "Hello,\n\nWe couldn't parse product IDs or quantities from your message. "
                "Please reply with product IDs (or exact product names) and quantities so we can process your order.\n\nBest regards,\nSupport Team"
            )
        })
        continue

    created = []
    out_of_stock = []
    for it in items:
        pid = it.get('product_id')
        pname = it.get('product_name')
        qty = int(it.get('quantity', 1))

        # Resolve name -> product_id if needed (simple contains match)
        if not pid and pname:
            match = products_df[products_df['name'].str.contains(pname, case=False, na=False)]
            if not match.empty:
                pid = str(match.iloc[0]['product_id'])

        if not pid:
            # unknown item: treat as out of stock / manual review
            order_status_rows.append({'email ID': eid, 'product ID': None, 'quantity': qty, 'status': 'out of stock'})
            out_of_stock.append((None, qty, 0))
            continue

        available = stock.get(pid, 0)
        if available >= qty:
            stock[pid] = available - qty
            order_status_rows.append({'email ID': eid, 'product ID': pid, 'quantity': qty, 'status': 'created'})
            created.append((pid, qty))
        else:
            order_status_rows.append({'email ID': eid, 'product ID': pid, 'quantity': qty, 'status': 'out of stock'})
            out_of_stock.append((pid, qty, available))

    # Craft professional response
    if created and not out_of_stock:
        lines = []
        for pid, q in created:
            name = product_lookup.get(pid, {}).get('name', pid)
            lines.append(f"- {name} (ID: {pid}) — qty {q}")
        resp_text = (
            "Hello,\n\nThank you — we have created an order for the following items:\n"
            + "\n".join(lines) +
            "\n\nWe will send tracking details once your order ships.\n\nBest regards,\nCustomer Support"
        )
    else:
        parts = []
        if created:
            for pid, q in created:
                name = product_lookup.get(pid, {}).get('name', pid)
                parts.append(f"- {name} (ID: {pid}) — qty {q} (created)")
        if out_of_stock:
            for pid, q, av in out_of_stock:
                name = product_lookup.get(pid, {}).get('name', 'Unknown product') if pid else 'Unknown product'
                parts.append(f"- {name} (ID: {pid}) — requested {q}, available {av} (out of stock)")
        resp_text = (
            "Hello,\n\nOrder update:\n"
            + "\n".join(parts)
            + "\n\nFor out-of-stock items you can wait for restock or request alternatives. Reply and we will help.\n\nBest regards,\nCustomer Support"
        )

    order_response_rows.append({'email ID': eid, 'response': resp_text})
    time.sleep(0.2)  # be polite with token quota


In [26]:
!pip install faiss-cpu --no-cache-dir
!pip install faiss-cpu==1.7.4 --no-cache-dir
import faiss


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 112.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-cpu==1.7.4 (from versions: 1.8.0, 1.8.0.post1, 1.9.0, 1.9.0.post1, 1.10.0, 1.11.0, 1.11.0.post1, 1.12.0, 1.13.0)
ERROR: No matching distribution found for faiss-cpu==1.7.4


# **Task 3. Handle product inquiry**

In [28]:
# Task 3 (FAISS-free) — safer: coerce subject/message to strings and handle missing values
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch
import time

embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Build embeddings for all products (run once)
product_texts = (
    products_df['product_id'].astype(str) + " | " +
    products_df['name'].fillna('') + " | " +
    products_df['description'].fillna('')
).tolist()

product_embeddings = embed_model.encode(product_texts, convert_to_tensor=True)

def retrieve_products_for_query(query, k=5):
    query_emb = embed_model.encode(query, convert_to_tensor=True)
    scores = util.cos_sim(query_emb, product_embeddings)[0]   # cosine similarities
    topk = torch.topk(scores, k)
    indices = topk.indices.cpu().numpy()
    results = []
    for idx in indices:
        p = products_df.iloc[idx]
        results.append({
            'product_id': str(p['product_id']),
            'name': p['name'],
            'stock': int(p['stock']),
            'description': p['description']
        })
    return results

inquiry_rows = []
for _, row in emails_df.iterrows():
    eid = row.get('email_id')
    # Coerce to str (handles NaN)
    subj = str(row.get('subject') or '')
    msg = str(row.get('message') or '')
    # Decide category safely
    cat_row = email_classification_df.loc[email_classification_df['email ID']==eid, 'category'].values
    cat = cat_row[0] if len(cat_row) else classify_email_with_llm(subj, msg)
    if cat != "product inquiry":
        continue

    # Build query safely
    query = (subj + " " + msg).strip()
    if query == "":
        # If empty message/subject, skip or create a polite reply
        inquiry_rows.append({'email ID': eid, 'response': "Hello,\n\nWe didn't find a question in your message. Could you please let us know what product or information you need?\n\nThanks,\nSupport Team"})
        continue

    top_products = retrieve_products_for_query(query, k=5)

    context_lines = []
    for p in top_products:
        desc = str(p['description'] or '')
        context_lines.append(f"{p['name']} (ID: {p['product_id']}), stock {p['stock']}. {desc[:250]}")
    context = "\n".join(context_lines)

    prompt = (
        "You are a helpful customer support agent. Use the product context below to answer the customer's question "
        "briefly and professionally. If a product is not available, suggest alternatives from the context.\n\n"
        f"Context:\n{context}\n\n"
        f"Customer:\nSubject: {subj}\nBody: {msg}\n\n"
        "Response:"
    )

    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role":"user","content":prompt}],
        max_tokens=300,
        temperature=0.2
    )
    answer = _get_message_text(resp).strip()
    inquiry_rows.append({'email ID': eid, 'response': answer})
    time.sleep(0.2)

# Convert to dataframe and inspect
inquiry_response_df = pd.DataFrame(inquiry_rows)
display(inquiry_response_df.head(10))


,email ID,response
0,E003,"Hello David,\n\nThank you for reaching out. Bo..."
1,E005,"Good day,\n\nThank you for your inquiry about ..."
2,E006,"Hello Sam,\n\nThank you for considering our Ch..."
3,E009,"Hola,\n\nGracias por contactarnos. Actualmente..."
4,E011,"Hi there,\n\nThank you for reaching out with y..."
5,E012,"Hello,\n\nThank you for reaching out and shari..."
6,E015,"Good morning,\n\nThank you for reaching out. F..."
7,E016,"Hello Claire,\n\nThank you for reaching out. F..."
8,E020,"Hello Antonio,\n\nThank you for reaching out. ..."
9,E021,Thank you for reaching out and for being a val...


In [30]:
# Convert to DataFrames
order_status_df = pd.DataFrame(order_status_rows)
order_response_df = pd.DataFrame(order_response_rows)
inquiry_response_df = pd.DataFrame(inquiry_rows)

# Save a local Excel copy IN COLAB (use /content, not /mnt/data)
import os
OUT_XLSX = "/content/ai_solution_output.xlsx"

with pd.ExcelWriter(OUT_XLSX, engine='openpyxl') as writer:
    email_classification_df.to_excel(writer, sheet_name='email-classification', index=False)
    order_status_df.to_excel(writer, sheet_name='order-status', index=False)
    order_response_df.to_excel(writer, sheet_name='order-response', index=False)
    inquiry_response_df.to_excel(writer, sheet_name='inquiry-response', index=False)

print("Saved local output:", OUT_XLSX)

# Write into the Google Sheet you already created (output_document)
from gspread_dataframe import set_with_dataframe

try:
    def set_sheet_df(title, df):
        # delete existing sheet if present
        try:
            ws = output_document.worksheet(title)
            output_document.del_worksheet(ws)
        except Exception:
            pass
        ws = output_document.add_worksheet(
            title=title,
            rows=str(max(100, len(df)+10)),
            cols=str(max(2, len(df.columns)))
        )
        set_with_dataframe(ws, df)

    set_sheet_df('email-classification', email_classification_df)
    set_sheet_df('order-status', order_status_df)
    set_sheet_df('order-response', order_response_df)
    set_sheet_df('inquiry-response', inquiry_response_df)

    output_document.share(None, perm_type='anyone', role='reader')
    print("Google Sheet updated. Link: https://docs.google.com/spreadsheets/d/" + output_document.id)

except Exception as e:
    print("Failed to update google sheet:", e)


Saved local output: /content/ai_solution_output.xlsx
Google Sheet updated. Link: https://docs.google.com/spreadsheets/d/1jlEYV-PyjxJ_gjRBPVSYuoxTAM5SO9BtE-1Wq5ExiSM
